0. Import module, library

In [135]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import re
import os
import io
import time
import random

import seaborn # Attention 시각화를 위해 필요!

print(tf.__version__)

2.6.0


# 1. 데이터 다운로드

In [136]:
workspace_path = os.getenv('HOME')+'/aiffel/transformer_chatbot'
data_path = os.getenv('HOME')+'/aiffel/transformer_chatbot/data'

In [137]:
data = pd.read_csv("../data/ChatbotData.csv")
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [138]:
len(data)

11823

# 2. 데이터 정제

In [139]:
# 결측치 확인
data.isnull().sum()

Q        0
A        0
label    0
dtype: int64

In [140]:
# label 삭제
data.drop(['label'], axis=1, inplace=True)

In [141]:
data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [142]:
# 샘플 확인
for i in range(10):
    print('Q :', data['Q'][i])
    print('A :', data['A'][i])    
    print()

Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



In [143]:
# 중복 데이터 확인

data[data.duplicated(['Q'])]

,Q,A
196,고양이 키우고 싶어,가족들과 상의해보세요.
235,공시 준비 힘들어,잘 될 거예요.
1294,돈 벌고 싶어,많이 벌수록 좋아요.
1445,로또 번호 알려줘,알면 제가 하죠.
1481,마음이 울적해,거리를 걸어보세요.
...,...,...
11642,착해서 잘해주는 건지 좋아하는 건지,헷갈린다고 말해보세요.
11649,첫 눈에 반하는게 가능해?,당연히 가능하죠.
11658,첫사랑 생각나,지금의 사랑에 충실하세요.
11732,커플여행이 나을까 그냥 우리끼리 갈까?,저는 둘이 가는 게 좋아요.


In [144]:
data[data['Q']==data['Q'][11819]] # 'data['Q'][11819]]' 값이 동일한 것 찾기 

,Q,A
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.


In [145]:
# 중복제거 

data[data.duplicated(['A'])] # 11,823 행에서 -> 4,044 행으로 축소

,Q,A
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.
12,가끔 뭐하는지 궁금해,그 사람도 그럴 거예요.
18,가스불 켜놓고 나온거 같아,빨리 집에 돌아가서 끄고 나오세요.
...,...,...
11806,혼자만 설레고 혼자서 끝내는 짝사랑 그만할래.,맘고생 많았어요.
11807,화이트데이에 고백할까요?,선물을 주면서 솔직하고 당당하게 고백해보세요.
11809,확실히 날 좋아하는 걸 아는 남자랑 친구가 될 수 있을까?,그 사람을 위해서는 그러면 안돼요.
11816,회식하는데 나만 챙겨줘. 썸임?,호감이 있을 수도 있어요. 그렇지만 조금 더 상황을 지켜보세요.


In [146]:
data[data['A']==data['A'][3]] # 중복 확인

,Q,A
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.


In [147]:
data[data['A']==data['A'][6]] # 중복 확인

,Q,A
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.


In [148]:
# 'A'행 중복 데이터 추가 제거 

data.drop_duplicates(['A'], inplace=True)
len(data)

7779

In [149]:
# 'Q'행 중복 데이터 추가 제거

data.drop_duplicates(['Q'], inplace=True)
len(data)

7727

In [150]:
print('total data length :', len(data))

total data length : 7727


In [151]:
# 데이터 전처리

def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()                                  # 소문자로 변경하고 양쪽 끝 공백 제거
    sentence = re.sub(r'[" "]+', " ", sentence)                          # re.sub() 정규 표현식을 사용하여 문자열에 패턴 찾아 대체
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ0-9]+", " ", sentence) # re.sub() 함수를 사용하여, 영어, 특수문자, 한글, 숫자를 제외한 모든 문자를 공백으로 대체
    sentence = sentence.strip()                                          # 이전 공백으로 대체한 공백을 다시 제거
    
    return sentence # 깔끔하게 전처리 완료

In [152]:
# 깔끔하게 전처리되고 Q, A 길이까지 정리된 모습

questions = [preprocess_sentence(q) for q in data['Q']]
answers = [preprocess_sentence(a) for a in data['A']]

print('questions data size :', len(questions))
print('answers data size :', len(answers))
print()
for i in range(10):
    print('Q :', questions[i])
    print('A :', answers[i])    
    print()

questions data size : 7727
answers data size : 7727

Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : ppl 심하네
A : 눈살이 찌푸려지죠.

Q : sd카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : sns 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : sns 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : sns보면 나만 빼고 다 행복해보여
A : 자랑하는 자리니까요.

Q : 가끔 궁금해
A : 그 사람도 그럴 거예요.

Q : 가끔은 혼자인게 좋다
A : 혼자를 즐기세요.



In [153]:
# 파일 저장
# 텍스트 파일을 생성하고, 주어진 말뭉치를 기록하는 역할을 함.
# 텍스트 파일('lang')을 생성하고, 주어진 말뭉치('corpus')를 기록.
# 파일의 이름은 {data_dir}/{lang}.txt 형식을 따름.


def make_txt(corpus, lang):
    with open(f'{data_dir}/{lang}.txt', 'w') as f:  
        # 파일을 쓰기모드('w'), data_dir 디렉토리 내에 위치, 파일 이름은 주어진 언어('lang')에 따라 결정
        for sen in corpus:
            f.write(str(sen) + '\n')
            # 각 문장(sen)을 문자열로 변환하고(str(sen)) 파일에 쓴 후 줄바꿈 문자('\n') 추가하여 다음 문장에 기록 
    f.close()
    print(f.name)
    return f.name

In [154]:
# 이걸 왜 만들었지?!!

# enc_path = make_txt(enc_corpus, lang='kor')
# dec_path = make_txt(dec_corpus, lang='eng')

# 3. 데이터 토큰화

In [155]:
## 토큰화와 패딩 작업을 수행 → 자연어 문장을 모델이 이해하고 처리할 수 있는 형태로 변환 

from konlpy.tag import Mecab                                          


def tokenize(corpus, vocab_size=50000):
    mecab = Mecab() # Mecab, 한국어 형태소 분석기         
    morph = [" ".join(mecab.morphs(sen)) for sen in corpus] 
    # Mecab의 'morphs' 메소드를 사용해 형태소 분석 수행 그 결과를 공백으로 연결하여 하나의 문자열로 

    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',
                                                     num_words=vocab_size)
    # keras의 Tokenizer 클래스를 사용하여 단어 토큰화 수행
    # 'num_words'는 토큰화 과정에서 사용할 최대 단어의 개수 설정

    tokenizer.fit_on_texts(morph)
    # 토큰화 해야하는 문장에 대해 Tokenizer 객체를 학습

    tensor = tokenizer.texts_to_sequences(morph)
    # Tokenizer의 'text_to_sequeces' 메소드를 사용하여 각 문장을 정수 시퀀스로 변환
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    # 'ped_sequences' 메소드를 사용하여 각 시퀀스(문장)의 길이를 동일하게 맞춰줌.
    # padding = 'post'는 패딩을 시퀀스 뒤쪽에 추가

        
    return tensor, tokenizer, morph
    # 변환된 텐서, 토큰화에 사용된 tokenizer, 형태소 분석 결과 반환

# 4. 데이터 증강

In [156]:
data_path

'/aiffel/aiffel/transformer_chatbot/data'

In [157]:
w2v_path = data_path + '/ko.bin'

In [158]:
# gensim 불러오기가 계속 오류나서 시간 뻇김


# from gensim.models.word2vec import Word2Vec
# wv_model = Word2Vec.load(w2v_path)

In [159]:
from gensim.models.keyedvectors import Word2VecKeyedVectors
wv_model = Word2VecKeyedVectors.load('/aiffel/data/word2vec_ko.model')

In [160]:
wv_model.wv.most_similar(positive='하루', topn=10)

[('매일', 0.6635749936103821),
 ('아침', 0.6332581639289856),
 ('시간', 0.6326238512992859),
 ('퇴근', 0.6313952803611755),
 ('보름', 0.6304587125778198),
 ('열흘', 0.626101553440094),
 ('반년', 0.6156684160232544),
 ('반나절', 0.6124287247657776),
 ('밤', 0.6106317043304443),
 ('이틀', 0.6030983924865723)]

In [161]:
# 단어 벡터 변환

wv_model.wv['하루']

array([ 3.4107115 , -1.1983304 ,  3.0812438 ,  2.2242346 , -0.17618921,
       -2.1063247 ,  2.2153187 ,  0.8750669 , -0.51959753, -0.5217767 ,
       -1.6041174 ,  0.0489326 ,  0.10208491, -3.3342006 , -2.1667967 ,
        1.4122443 ,  1.2402686 ,  0.16979106,  1.8354017 ,  1.5598876 ,
       -2.4736938 ,  1.2335609 ,  0.32076317, -0.6427263 , -1.5151329 ,
       -1.0098913 , -1.0104748 , -0.28930202,  2.35188   , -0.6010162 ,
       -2.1898339 ,  2.0393732 , -0.4712875 ,  0.44534144,  3.380081  ,
       -0.0361988 ,  0.7425784 ,  0.35870308, -2.0619228 ,  1.6251751 ,
        0.751604  ,  0.7271253 , -1.8895692 , -0.33305213, -2.7046776 ,
       -0.88320017, -0.91837466, -1.0607344 ,  3.1014934 , -0.47572172,
        0.7655098 ,  1.2335111 ,  0.15673797,  0.5468275 , -1.4003606 ,
       -0.23191464,  0.5468947 ,  2.016602  ,  2.2702746 ,  3.1519432 ,
        0.16185166,  0.34010816, -1.1395248 , -2.2745152 ,  1.073325  ,
       -2.7888043 , -0.38226172, -0.6075256 ,  4.0218983 , -2.07

In [162]:
# 유사한 단어 출력(코사인 유사도)
wv_model.wv.most_similar('공부')

[('중퇴', 0.7482234835624695),
 ('전공', 0.728223979473114),
 ('자퇴', 0.7157790660858154),
 ('졸업', 0.7004087567329407),
 ('가르치', 0.6782639622688293),
 ('수료', 0.6718839406967163),
 ('수학', 0.6572521328926086),
 ('유학', 0.6553082466125488),
 ('입학', 0.6530455350875854),
 ('집필', 0.6526979207992554)]

In [163]:
# 단어장에 단어가 있는지 확인해보기 

for tok in sample_tokens:
    if tok is selected_tok:
        if tok in wv_model.wv:  # 단어장에 단어가 있는지 확인
            result += wv_model.wv.most_similar(tok)[0][0] + " "
        else:
            result += tok + " "  # 단어장에 없는 경우, 원래 단어 사용
    else:
        result += tok + " "

print(sample_tokens)

['가끔', '궁금해']


In [172]:
import random

sample_sentence = "가끔 궁금해"
sample_tokens = sample_sentence.split()

selected_tok = random.choice(sample_tokens)

result = ""
for tok in sample_tokens:
    if tok is selected_tok:
        result += wv_model.wv.most_similar(tok)[0][0] + " "

    else:
        result += tok + " "

print("From:", sample_sentence)
print("To:", result)

From: 가끔 궁금해
To: 이따금 궁금해 


In [165]:
# 입력된 문장('sentence')의 임의의 단어를 해당 단어와 가장 유사한 단어로 바꾸는 
# "lexical substitution(어휘 대체)" 작업을 수행하는 함수
# 이 수행은 데이터 증강(data augmentation)의 한 방법으로, 텍스트를 다양하게 만들어 
# 모델의 일반화 성능을 향상시키는 데 도움이 됨.

def lexical_sub(sentence, word2vec):
    import random

    res = ""  # 결과가 저장
    toks = sentence.split()  # 입력된 문장을 공백을 기준으로 나눠 각 단어를 'toks'라는 리스트에 저장

    try:  
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]
        # 이 부분에서는 'toks'에서 임의의 단어를 선택하여 그 단어와 가장 유사한 단어를 찾는 역할 
        # 없으면 'None'을 반환하고 함수를 종료
        
    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "
        # 각 단어에 대해 반본 작업을 수행, 현재 단어가 선택된 단어와 같다면, 
        # 단어를 가장 유사한 단어로 바꿔 'res'에 추가 
        # 현재 단어가 선택된 단어와 다르면, 원래 단어를 'res'를 반환
            
    return res
    # 마지막으로 수정된 문장 'res'를 반환

In [166]:
# 원래의 말뭉치('src_corpus'와 'tgt_corpus')를 받아서, 이를 'lexical substitution' 기법인 'lexical_sub' 함수를 
# 사용해 데이터 증강(data augmentation)을 수행하고, 증강된 말뭉치('new_src_corpus'와 'new_tgt_corpus')를 반환


from tqdm import tqdm_notebook

def augment_corpus(src_corpus, tgt_corpus, wv):
    new_src_corpus = []  # 증강된 문장들을 담을 새로운 말뭉치를 빈 리스트로 초기화
    new_tgt_corpus = []  # 증강된 문장들을 담을 새로운 말뭉치를 빈 리스트로 초기화
    corpus_size = len(src_corpus)
    
    for i in tqdm_notebook(range(corpus_size)):  
        # 'tqdm_notebook'을 사용해 전체 말뭉치 크기('corpus_size')만큼 반복문을 돌며, 
        # 각 문장에 대해 데이터 증강을 수행, 시각적으로 보여주는데 도움이 됨.
        
        old_src = src_corpus[i]
        old_tgt = tgt_corpus[i]
        # 현재 인덱스(i)에 해당하는 원래의 소스 문장('old_src')과 타겟 문장('old_tgt')를 가져온다. 

        new_src = lexical_sub(old_src, wv)
        new_tgt = lexical_sub(old_tgt, wv)
        # 원래의 소스문장과 타겟문장을 'lexical_sub' 함수에 적용해 증강된 문장(new_src와 new_tgt)을 생성

        if new_src: 
            new_src_corpus.append(new_src)
            new_tgt_corpus.append(old_tgt)
            
        if new_tgt: 
            new_src_corpus.append(old_src)
            new_tgt_corpus.append(new_tgt)
        #  증강된 소스 문장(new_src)나 타겟 문장(new_tgt)이 None이 아니라면 (즉, 증강이 성공적으로 이루어졌다면), 
        # 이를 새로운 말뭉치(new_src_corpus와 new_tgt_corpus)에 추가
        # 소스 문장이 증가되었을 경우, 원래의 타겟 문장을, 타겟 문장이 증강되었을 경우 원래의 소스 문장을 쌍으로 유지    

    # print(new_corpus[:10])
    return new_src_corpus, new_tgt_corpus
    # 증강된 소스 말충치와 타겟 말뭉치를 반환

In [167]:
# 'tokenize' 함수는 입력된 말뭉치('questions' 또는 'answers')를 토크나이징(단어 단위로 분할)하고,
# 이를 숫자로 변환하는 작업을 수행, '_'는 반환값을 무시하겠다는 의미
# 이 코드는 questions와 answers 말뭉치를 토크나이징하여 각각 src_corpus_tokenized와 tgt_corpus_tokenized에 저장

_, _, src_corpus_tokenized = tokenize(questions)
# 'questions' 말뭉치를 토크나이징한 결과인 'src_corpus_tokenized'를 얻는다. 

_, _, tgt_corpus_tokenized = tokenize(answers)
# 'answers' 말뭉치를 토크나이징한 결과인 'tgt_corpus_tokenized'를 얻는다. 


In [168]:
src_corpus_tokenized[:10]

['12 시 땡 !',
 '1 지망 학교 떨어졌 어',
 '3 박 4 일 놀 러 가 고 싶 다',
 'ppl 심하 네',
 'sd 카드 망가졌 어',
 'sns 맞 팔 왜 안 하 지 ㅠㅠ',
 'sns 시간 낭비 인 거 아 는데 매일 하 는 중',
 'sns 보 면 나 만 빼 고 다 행복 해 보여',
 '가끔 궁금 해',
 '가끔 은 혼자 인 게 좋 다']

In [169]:
tgt_corpus_tokenized[:10]

['하루 가 또 가 네요 .',
 '위로 해 드립니다 .',
 '여행 은 언제나 좋 죠 .',
 '눈살 이 찌푸려 지 죠 .',
 '다시 새로 사 는 게 마음 편해요 .',
 '잘 모르 고 있 을 수 도 있 어요 .',
 '시간 을 정하 고 해 보 세요 .',
 '자랑 하 는 자리 니까요 .',
 '그 사람 도 그럴 거 예요 .',
 '혼자 를 즐기 세요 .']

In [170]:
# augment_corpus 함수를 사용하여 src_corpus_tokenized와 tgt_corpus_tokenized 데이터에 대해 
# 'lexical substitution' 기법을 이용한 데이터 증강을 수행

new_src_corpus, new_tgt_corpus = augment_corpus(src_corpus_tokenized, tgt_corpus_tokenized, wv_model)
# 'augment_corpus' 함수에 토크나이징된 원본 말뭉치(src_corpus_tokenized, tgt_corpus_tokenized)와 
# Word2Vec('mv_model')을 인자로 전달하고, 증강된 말뭉치('new_scr_corpus', 'new_tgt_corpus')를 반환한다.

/tmp/ipykernel_38/2144691072.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(corpus_size)):


  0%|          | 0/7727 [00:00<?, ?it/s]

In [171]:
print(len(new_src_corpus), len(new_tgt_corpus))

0 0
